# Modeling
In this notebook, we'll be modeling the data we've previously prepared. Out notebook will be laid out as follows:

1. Model Selection & Generation
2. Hyperparameter Optimization
3. Fine-Tuning (if needed)
4. Reporting Best Model(s) + Settings
5. Interpretation
6. Conclusion

Our eventual goal here is two-fold:

1. Accurately [and fairly] model the diabetes dataset
2. Interpret the results to find something worth recommending to those wanting to reduce risk of diabetes. This can be via LIME/SHAP (i.e. some interpretable model that approximates the neural network) or via analyzing a more simple model's structure (i.e. regression coefficients, random forest decision boundaries)

In [1]:
# Environment Setup
from utils.model import *
from utils.dataset import *

***
## Model Selection & Generation

In [2]:
# generate lookup for models
models = {
    # "tree": TreeClassifier(target="diabetes", path="../datasets/pre_split_processed.parquet"),
    "ffnn": MLPClassifier(target="diabetes", path="../datasets/pre_split_processed.parquet")
}

# manual search
models["ffnn"].set_hyperparams({
    "learning_rate": .0001,
    "batch_size": 128,
    "num_hidden": 4,
    "hidden_size": 1024,
    "num_epochs": 50,
    "classify_fn": "sigmoid"
})

# train & test basic model
for mt, model in models.items():
    # attempt to load, else train and test
    # if not model.load_model():
    model.train_model(verbose=1)
    model.test_model()

<Train-Test Split Report>
Train: 512724 obs, 170908 no diabetes [0], 170908 pre-diabetes [1], 170908 diabetes [2]
Test: 50736 obs, 42795 no diabetes [0], 944 pre-diabetes [1], 6997 diabetes [2]


100%|██████████| 4006/4006 [00:27<00:00, 147.82it/s]


Epoch 1/50, Loss: 0.9313979456559218


100%|██████████| 4006/4006 [00:26<00:00, 148.63it/s]


Epoch 2/50, Loss: 0.8206022908344783


...

Epoch 48/50, Loss: 0.6558808735234702


100%|██████████| 4006/4006 [00:26<00:00, 149.38it/s]


Epoch 49/50, Loss: 0.6554071685402499


100%|██████████| 4006/4006 [00:26<00:00, 149.14it/s]


Epoch 50/50, Loss: 0.6558454898346441

<Test Report>
Precision: [no diabetes] 0.8474442612373647, [pre-diabetes] 0.15384615384615385, [diabetes] 0.024390243902439025
Recall: [no diabetes] 0.8286715737819839, [pre-diabetes] 0.17921966557095897, [diabetes] 0.019067796610169493
F1-Score: [no diabetes] 0.8379527893953357, [pre-diabetes] 0.16556641140744652, [diabetes] 0.02140309155766944
Support: [no diabetes] 42795, [pre-diabetes] 6997, [diabetes] 944
Accuracy: 72.4042%


***
## Hyperparameter Optimization

In [3]:
# optimize hyperparams
optimizer_results = {model_type: model.optimize_hyperparams(kfold=2) for model_type, model in models.items()}
print(optimizer_results)

<Grid-Search>
Testing 7 combinations WITHOUT cross-validation


IndexError: tuple index out of range

***
## Fine-Tuning + Other Adjustments

***
## Best Model Report

***
## Interpretation

***
## Conclusion